In [9]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Read the specific sheet "data" from the Excel file
df = pd.read_excel("basemaps/plan_distrito_verde.xlsx", sheet_name="data")

print("Columnas disponibles:")
print(df.columns.tolist())
print(f"\nDimensiones iniciales: {df.shape}")

# Function to parse coordinates from "lat, lon" to Point(lon, lat)
def parse_coordinates(coord_str):
    if pd.isna(coord_str) or coord_str == '':
        return None
    try:
        # Parse "lat, lon" and convert to Point(lon, lat) for GeoJSON standard
        lat, lon = map(float, str(coord_str).strip().split(','))
        return Point(lon, lat)
    except Exception as e:
        print(f"Error parsing coordinate: {coord_str} - {e}")
        return None

# Check if 'coordinates' column exists
if 'coordinates' in df.columns:
    print("\nProcesando columna 'coordinates'...")
    df['geometry'] = df['coordinates'].apply(parse_coordinates)
else:
    print("\n¡Advertencia! Columna 'coordinates' no encontrada.")
    print("Columnas disponibles:", df.columns.tolist())
    df['geometry'] = None

# Create GeoDataFrame with EPSG:4326 (WGS84)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

# Filter rows with valid geometry
valid_gdf = gdf[gdf['geometry'].notna()].copy()

print(f"\nGeoDataFrame creado:")
print(f"  - Total de filas: {len(gdf)}")
print(f"  - Filas con geometría válida: {len(valid_gdf)}")
print(f"  - CRS: {valid_gdf.crs}")

# Display first few rows
print("\nPrimeras filas del GeoDataFrame:")
valid_gdf.head()

Columnas disponibles:
['tipo_jornada', 'fecha_actividad', 'hora_encuentro', 'direccion', 'coordinates', 'objetivo_actividad', 'personas_requeridas_grupo', 'grupos_requeridos', 'lider_actividad', 'telefono', 'observaciones']

Dimensiones iniciales: (22, 11)

Procesando columna 'coordinates'...

GeoDataFrame creado:
  - Total de filas: 22
  - Filas con geometría válida: 21
  - CRS: EPSG:4326

Primeras filas del GeoDataFrame:


,tipo_jornada,fecha_actividad,hora_encuentro,direccion,coordinates,objetivo_actividad,personas_requeridas_grupo,grupos_requeridos,lider_actividad,telefono,observaciones,geometry
0,Recorrido en área protegida,2026-01-08,10:00:00,"Ecoparque pisamos, Carrera 26 c # 73 a Marroquin","3.440099831592101, -76.48207996004042",Realizar recorridos y presencia en el ecoparqu...,15,REACCIÓN,Lina Botia,3152569980,Se programa por Calendar a los grupos,POINT (-76.48208 3.4401)
1,IEC,2026-01-08,02:00:00,"Calle Del Comercio, Estación De Policia Los Ma...","3.426918406198808, -76.48405190360928",Realizar recorridos y presencia en los estable...,15,REACCIÓN,Israel corredor,3152569980,Se programa por Calendar a los grupos,POINT (-76.48405 3.42692)
2,IEC,2026-01-09,20:00:00,"Zona nocturna calle 44 norte , Sede Alterna Da...","3.479283603312855, -76.51092280827497",Realizar recorridos y presencia en los estable...,20,REACCIÓN,Israel corredor,3152569980,Se programa por Calendar a los grupos,POINT (-76.51092 3.47928)
3,IVC,2026-01-16,20:00:00,"Zona del parque El Peñon , Estación de policia...","3.450095527742933, -76.54275126128167",Realizar intervención cumpliendo con actividad...,15,"ACUSTICA, REACCIÓN, RESIDUOS SOLIDOS",Israel corredor,313 4540578,Se programa por Calendar a los grupos,POINT (-76.54275 3.4501)
4,IVC,2026-01-17,20:00:00,"Zona de la carrera 66 entre 2 y 5 , Estación d...","3.415681596558856, -76.55309190360931",Realizar intervención cumpliendo con actividad...,15,"ACUSTICA, REACCIÓN, RESIDUOS SOLIDOS",Israel corredor,313 4540578,Se programa por Calendar a los grupos,POINT (-76.55309 3.41568)


In [12]:
valid_gdf.columns

Index(['tipo_jornada', 'fecha_actividad', 'hora_encuentro', 'direccion',
       'objetivo_actividad', 'personas_requeridas_grupo', 'grupos_requeridos',
       'lider_actividad', 'telefono', 'observaciones', 'geometry'],
      dtype='str')

In [11]:
valid_gdf.drop(columns=['coordinates'], inplace=True)

In [13]:
import firebase_admin
from firebase_admin import credentials, firestore
from datetime import datetime

# Initialize Firebase (skip if already initialized)
try:
    firebase_admin.get_app()
except ValueError:
    cred = credentials.Certificate("env/dagma-85aad-b7afe1c0f77f.json")
    firebase_admin.initialize_app(cred)

# Get Firestore client
db = firestore.client()

# Reference to the collection
collection_ref = db.collection("plan_distrito_verde")

# Iterate through valid_gdf and add documents
for idx, row in valid_gdf.iterrows():
    # Prepare the document data
    doc_data = {
        "tipo_jornada": str(row['tipo_jornada']) if pd.notna(row['tipo_jornada']) else "",
        "fecha_actividad": datetime.strptime(str(row['fecha_actividad']).split()[0], '%Y-%m-%d').strftime('%d/%m/%Y'),
        "hora_encuentro": str(row['hora_encuentro']),
        "punto_encuentro": {
            "direccion": str(row['direccion']) if pd.notna(row['direccion']) else "",
            "geometry": {
                "type": "Point",
                "coordinates": [row['geometry'].x, row['geometry'].y]
            }
        },
        "objetivo_actividad": str(row['objetivo_actividad']) if pd.notna(row['objetivo_actividad']) else "",
        "personas_requeridas_grupo": int(row['personas_requeridas_grupo']),
        "grupos_requeridos": [g.strip() for g in str(row['grupos_requeridos']).split(',')],
        "lider_actividad": str(row['lider_actividad']) if pd.notna(row['lider_actividad']) else "",
        "telefono": str(row['telefono']) if pd.notna(row['telefono']) else ""
    }
    
    # Add observaciones only if it's not NaN
    if pd.notna(row['observaciones']):
        doc_data["observaciones"] = str(row['observaciones'])
    
    # Add document to Firestore
    collection_ref.add(doc_data)
    print(f"Documento {idx} agregado exitosamente")

print(f"\n✅ Total de documentos agregados: {len(valid_gdf)}")

Documento 0 agregado exitosamente
Documento 1 agregado exitosamente
Documento 2 agregado exitosamente
Documento 3 agregado exitosamente
Documento 4 agregado exitosamente
Documento 5 agregado exitosamente
Documento 6 agregado exitosamente
Documento 7 agregado exitosamente
Documento 8 agregado exitosamente
Documento 9 agregado exitosamente
Documento 10 agregado exitosamente
Documento 11 agregado exitosamente
Documento 12 agregado exitosamente
Documento 13 agregado exitosamente
Documento 14 agregado exitosamente
Documento 15 agregado exitosamente
Documento 16 agregado exitosamente
Documento 17 agregado exitosamente
Documento 18 agregado exitosamente
Documento 20 agregado exitosamente
Documento 21 agregado exitosamente

✅ Total de documentos agregados: 21
